### This notebook is to convert DART-FISH data to more standard spatial data format (Xenium compatible).

In [60]:
import sopa
import spatialdata as sd
# import spatialdata_xenium_explorer
import os, numpy as np, pandas as pd, anndata
from matplotlib import pyplot as plt
from skimage.io import imread

In [61]:
# Load DART-FISH data
dir = '/home/xli/dartfish/cartilage_muscle/20250418_Cartilage/analysis/'
CellMask = imread(os.path.join(dir,"2_RegisteredMIPfirst/stitched/10_stain_ch03.tif"))
DAPI = imread(os.path.join(dir,"2_RegisteredMIPfirst/stitched/10_stain_ch00.tif"))
anc = imread(os.path.join(dir,"2_RegisteredMIPfirst/stitched/0_anchor_ch02.tif"))
WGA = imread(os.path.join(dir,"2_RegisteredMIPfirst/stitched/10_stain_ch01.tif"))

spots = pd.read_csv(os.path.join(dir,"4_CellAssignment_nuc_emptyProbabilityThr1_dist6.5_DAPIcellpose/spots_assigned_custom.tsv"), sep="\t", index_col=0)
mask = np.load(os.path.join(dir,"4_CellAssignment_nuc_emptyProbabilityThr1_dist6.5_DAPIcellpose/segmentation_mask_custom.npy"))

CellMask = (CellMask >> 4).astype(np.uint8)
DAPI = (DAPI >> 4).astype(np.uint8)
anc = (anc >> 4).astype(np.uint8)
WGA = (WGA >> 4).astype(np.uint8)

In [62]:
spots = spots.drop(['x', 'y'], axis=1).rename({'xg' : 'x', 'yg' : 'y'}, axis=1)
spots.head()

,x,y,z,label,area,weight_max,weight_mean,diameter,eccentricity,area_bbox,...,perimeter,feret_diameter_max,lasso_r2,ols_r2,target,gene,fov,EmptyProb,cell_label,dist2cell
0,384.7,254.1,0,FOV34_Krt15_00012002_6,74.0,0.831,0.312,9.707,0.639,99.0,...,32.728,11.705,0.318,0.610,Krt15_00012002,Krt15,FOV34,0.012,1,224.40
1,386.0,248.8,0,FOV34_Erfe_00303200_10,7.0,0.184,0.135,2.985,0.727,12.0,...,7.243,4.000,0.180,0.559,Erfe_00303200,Erfe,FOV34,0.535,1,219.54
2,607.9,167.1,0,FOV34_Runx2_03031000_6,33.0,0.193,0.136,6.482,0.572,42.0,...,18.485,7.616,0.191,0.418,Runx2_03031000,Runx2,FOV34,0.093,1,276.80
3,522.2,96.7,0,FOV34_H3f3b_00110003_8,63.0,0.236,0.139,8.956,0.875,104.0,...,34.763,14.318,0.199,0.433,H3f3b_00110003,H3f3b,FOV34,0.065,1,166.71
4,503.2,28.9,0,FOV34_Mbp_00200011_1,40.0,0.223,0.146,7.136,0.504,56.0,...,21.899,8.544,0.248,0.472,Mbp_00200011,Mbp,FOV34,0.118,1,125.45


In [63]:
spots_model = sd.models.PointsModel.parse(spots, feature_key='gene', instance_key='label') # , coordinates={'x' : 'xg', 'y' : 'yg'}

In [64]:
mask.shape

(12990, 9660)

In [65]:
# spatialdata.SpatialData(images={'DAPI' : DAPI, 'n9' : n9})
DAPI_im2d = sd.models.Image2DModel.parse(DAPI[None, ...])
# n9_im2d = sd.models.Image2DModel.parse(n9[None, ...])
CellMask_im2d = sd.models.Image2DModel.parse(CellMask[None, ...])
mask_l2d = sd.models.Labels2DModel.parse(mask)
composite = np.stack([DAPI, CellMask, WGA, anc])
composite_im2d = sd.models.Image2DModel.parse(composite)
# sd.Image2DModel()

INFO     no axes information specified in the object, setting `dims` to: ('c', 'y', 'x')                           
INFO     no axes information specified in the object, setting `dims` to: ('c', 'y', 'x')                           
INFO     no axes information specified in the object, setting `dims` to: ('y', 'x')                                
INFO     no axes information specified in the object, setting `dims` to: ('c', 'y', 'x')                           


In [93]:
from anndata import AnnData
import scanpy as sc
DF_celllocs = pd.read_csv(os.path.join(dir, "4_CellAssignment_nuc_emptyProbabilityThr1_dist6.5_DAPIcellpose/cell_info_custom.tsv"), sep = "\t", index_col='cell_label')
asg_spots = pd.read_csv(os.path.join(dir, "4_CellAssignment_nuc_emptyProbabilityThr1_dist6.5_DAPIcellpose/spots_assigned_custom.tsv"), sep='\t', index_col=0)
print(asg_spots.shape)
asg_spots = asg_spots.loc[~asg_spots['gene'].str.startswith('Empty')] #.query("gene != 'Empty'")
print(asg_spots.shape)
dist2cell_thr = 500
asg_spots2 = asg_spots.query("dist2cell <= @dist2cell_thr")
print(asg_spots2.shape)
# asg_spots2 = asg_spots2.loc[~((asg_spots2['gene'].isin(['MBP', 'MOBP'])) & (asg_spots2['dist2cell'] > dist2cell_thr_mbp))]
# print(asg_spots2.shape)

cbg2 = asg_spots2[['cell_label', 'gene']].groupby(by = ['cell_label', 'gene']).size()
cbg2 = cbg2.reset_index().pivot(index = 'cell_label', columns = 'gene').fillna(0).astype(int)
cbg2.columns = cbg2.columns.droplevel()

print(cbg2.shape)
min_gcount = 0
cbg2 = cbg2[cbg2.columns[(cbg2.sum(axis=0) >= min_gcount)]]
cell_labels = [str(x) for x in cbg2.index]
print(cbg2.shape)

df_ad = AnnData(X = cbg2.to_numpy(), obs=pd.DataFrame(index=cbg2.index),# dtype=float, 
                   obsm={'spatial': DF_celllocs.loc[cbg2.index, ['centroid_x', 'centroid_y']].to_numpy(),
                         'cell_area': DF_celllocs.loc[cbg2.index, ['area']].to_numpy()},
                var = pd.DataFrame(index=cbg2.columns.to_numpy()))
df_ad.layers['raw'] = df_ad.X.copy()
df_ad.layers["sqrt_norm"] = np.sqrt(sc.pp.normalize_total(df_ad, inplace=False)["X"])

print(df_ad.shape)

from copy import deepcopy

t = deepcopy(df_ad)
t.varm = None
os.makedirs('../analysis/xeniumX_DAPICellpose',exist_ok=True)
t.write_h5ad("../analysis/xeniumX_DAPICellpose/mCartilage.h5ad")

(8820200, 24)
(8596608, 24)
(8596608, 24)
(11982, 306)
(11982, 306)
(11982, 306)


/home/xli/.local/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [67]:
asg_spots.head()

,xg,yg,z,y,x,label,area,weight_max,weight_mean,diameter,...,perimeter,feret_diameter_max,lasso_r2,ols_r2,target,gene,fov,EmptyProb,cell_label,dist2cell
0,384.7,254.1,0,254.1,37.3,FOV34_Krt15_00012002_6,74.0,0.831,0.312,9.707,...,32.728,11.705,0.318,0.610,Krt15_00012002,Krt15,FOV34,0.012,1,224.40
1,386.0,248.8,0,248.8,38.6,FOV34_Erfe_00303200_10,7.0,0.184,0.135,2.985,...,7.243,4.000,0.180,0.559,Erfe_00303200,Erfe,FOV34,0.535,1,219.54
2,607.9,167.1,0,167.1,260.5,FOV34_Runx2_03031000_6,33.0,0.193,0.136,6.482,...,18.485,7.616,0.191,0.418,Runx2_03031000,Runx2,FOV34,0.093,1,276.80
3,522.2,96.7,0,96.7,174.8,FOV34_H3f3b_00110003_8,63.0,0.236,0.139,8.956,...,34.763,14.318,0.199,0.433,H3f3b_00110003,H3f3b,FOV34,0.065,1,166.71
4,503.2,28.9,0,28.9,155.8,FOV34_Mbp_00200011_1,40.0,0.223,0.146,7.136,...,21.899,8.544,0.248,0.472,Mbp_00200011,Mbp,FOV34,0.118,1,125.45


In [68]:
# df_ad = anndata.read_h5ad("./anndata_objects/240314_7mKid_renameAnnot.h5ad")
df_ad = anndata.read_h5ad("../analysis/xeniumX_DAPICellpose/mCartilage.h5ad")
df_ad.obs['cell_id'] = df_ad.obs.index.values
df_ad.obs['regkey'] = 'mCartilage'

# for ob in ['l1_final1', 'l2_normal1', 'l2_weird1']:
#     df_ad.obs[ob] = df_ad.obs[ob].cat.add_categories('Unassigned')
#     df_ad.obs.loc[df_ad.obs[ob].isna(), ob] = 'Unassigned'

In [69]:
df_ad

AnnData object with n_obs × n_vars = 11982 × 306
    obs: 'cell_id', 'regkey'
    obsm: 'cell_area', 'spatial'
    layers: 'raw', 'sqrt_norm'

In [70]:
sd.models.TableModel.parse(df_ad, region = 'mCartilage', region_key='regkey', instance_key = 'cell_id')

/home/xli/.conda/envs/sopav2/lib/python3.10/site-packages/spatialdata/models/models.py:1053: UserWarning: Converting `region_key: regkey` to categorical dtype.
  return convert_region_column_to_categorical(adata)


AnnData object with n_obs × n_vars = 11982 × 306
    obs: 'cell_id', 'regkey'
    uns: 'spatialdata_attrs'
    obsm: 'cell_area', 'spatial'
    layers: 'raw', 'sqrt_norm'

In [71]:
from shapely.geometry import Polygon
from skimage.measure import regionprops
import geopandas
labs, geoms = [], []
for reg in regionprops(mask.T):
    geoms.append(Polygon(reg.coords).convex_hull)
    labs.append(reg.label)

In [84]:
poly_gp = geopandas.GeoDataFrame({'geometry' : geoms})
poly_gp = poly_gp.set_index(np.array(labs, dtype=str))
cell_to_remove = []
if len(poly_gp[poly_gp.geometry.type != 'Polygon']) > 0:
    cell_to_remove = poly_gp[poly_gp.geometry.type != 'Polygon'].index.to_list()
    poly_gp = poly_gp[poly_gp.geometry.type == 'Polygon']
    
    # poly_gp= poly_gp.set_index(np.array(np.arange(len(poly_gp))+1, dtype=str))
poly_gp = poly_gp[poly_gp.index.isin(cell_labels)]
poly_gp 

,geometry
1,"POLYGON ((350 0, 348 2, 348 30, 349 32, 360 31..."
2,"POLYGON ((2367 0, 2364 1, 2363 2, 2358 9, 2358..."
3,"POLYGON ((2404 88, 2402 89, 2398 93, 2399 95, ..."
4,"POLYGON ((2398 94, 2388 95, 2385 98, 2385 106,..."
5,"POLYGON ((6351 145, 6341 175, 6341 177, 6345 1..."
...,...
11979,"POLYGON ((7710 12911, 7707 12913, 7706 12914, ..."
11980,"POLYGON ((8992 12942, 8990 12943, 8986 12947, ..."
11981,"POLYGON ((8958 12951, 8956 12952, 8952 12955, ..."
11982,"POLYGON ((9293 12958, 9272 12972, 9269 12975, ..."


In [85]:
mask_s2d = sd.models.ShapesModel.parse(poly_gp)
mask_s2d

,geometry
1,"POLYGON ((350 0, 348 2, 348 30, 349 32, 360 31..."
2,"POLYGON ((2367 0, 2364 1, 2363 2, 2358 9, 2358..."
3,"POLYGON ((2404 88, 2402 89, 2398 93, 2399 95, ..."
4,"POLYGON ((2398 94, 2388 95, 2385 98, 2385 106,..."
5,"POLYGON ((6351 145, 6341 175, 6341 177, 6345 1..."
...,...
11979,"POLYGON ((7710 12911, 7707 12913, 7706 12914, ..."
11980,"POLYGON ((8992 12942, 8990 12943, 8986 12947, ..."
11981,"POLYGON ((8958 12951, 8956 12952, 8952 12955, ..."
11982,"POLYGON ((9293 12958, 9272 12972, 9269 12975, ..."


In [87]:
cell_to_remove

[]

In [88]:
# remove cell id that are not polygon
df_ad = df_ad[~df_ad.obs["cell_id"].isin(cell_to_remove)]
# df_ad = df_ad[df_ad.obs["cell_id"].isin(cell_labels)]

# df_ad.obs = df_ad.obs.loc[~df_ad.obs['cell_id'].isin(cell_to_remove)]
spots_model = spots_model[~spots_model['cell_label'].isin(cell_to_remove)]

In [89]:
df_ad.obs.loc[df_ad.obs['cell_id'].isin(cell_to_remove)]

,cell_id,regkey
cell_label,,


In [90]:

df_sd2 = sd.SpatialData(images={'DAPI' : DAPI_im2d,'CellMask' : CellMask_im2d, 'composite' : composite_im2d}, 
                        shapes = {'mCartilage':poly_gp}, 
                        points={'spots' : spots_model}
                        , tables=df_ad)
df_sd2.write("../analysis/DARTFISH.zarr",overwrite=True)

INFO     The Zarr backing store has been changed from None the new file path: ../analysis/DARTFISH.zarr            


In [81]:
df_sd2

SpatialData object, with associated Zarr store: /home/projects/dartfish/cartilage_muscle/20250418_Cartilage/analysis/DARTFISH.zarr
├── Images
│     ├── 'CellMask': DataArray[cyx] (1, 12990, 9660)
│     ├── 'DAPI': DataArray[cyx] (1, 12990, 9660)
│     └── 'composite': DataArray[cyx] (4, 12990, 9660)
├── Points
│     └── 'spots': DataFrame with shape: (<Delayed>, 22) (3D points)
├── Shapes
│     └── 'mCartilage': GeoDataFrame shape: (11983, 1) (2D shapes)
└── Tables
      └── 'table': AnnData (11982, 306)
with coordinate systems:
    ▸ 'global', with elements:
        CellMask (Images), DAPI (Images), composite (Images), spots (Points), mCartilage (Shapes)

In [92]:
# import spatialdata
# import sopa
sopa.io.explorer.write(
    "../analysis/xeniumX_DAPICellpose/", df_sd2, image_key='composite', shapes_key='mCartilage', points_key='spots', gene_column='gene', pixel_size=0.273738591762728, 
                                  layer='raw',polygon_max_vertices=13, ram_threshold_gb=1, mode=None)

[INFO] (sopa.io.explorer.table) Writing table with 306 columns
[INFO] (sopa.io.explorer.table) Writing 1 cell categories: regkey
[INFO] (sopa.io.explorer.shapes) Writing 11982 cell polygons
[INFO] (sopa.io.explorer.points) Writing 8820200 transcripts
[INFO] (sopa.io.explorer.points)    > Level 0: 8820200 transcripts
[INFO] (sopa.io.explorer.points)    > Level 1: 2205050 transcripts
[INFO] (sopa.io.explorer.points)    > Level 2: 551262 transcripts
[INFO] (sopa.io.explorer.points)    > Level 3: 137815 transcripts
[INFO] (sopa.io.explorer.points)    > Level 4: 34453 transcripts
[INFO] (sopa.io.explorer.images) Writing multiscale image with procedure=semi-lazy (load in memory when possible)
[INFO] (sopa.io.explorer.images)    (Loading image of shape (4, 12990, 9660)) in memory
[INFO] (sopa.io.explorer.images)    > Image of shape (4, 12990, 9660)
[INFO] (sopa.io.explorer.images)    > Image of shape (4, 6495, 4830)
[INFO] (sopa.io.explorer.images)    > Image of shape (4, 3247, 2415)
[INFO] (